In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#langchain library
!pip3 install langchain==0.1.0
#LLM
!pip3 install accelerate==0.25.0
!pip3 install transformers==4.36.2
!pip3 install bitsandbytes==0.41.2
#Text Embedding
!pip3 install sentence-transformers==2.2.2
!pip3 install InstructorEmbedding==1.0.1
#vectorstore
!pip3 install pymupdf==1.23.8
!pip3 install faiss-gpu==1.7.2
!pip3 install faiss-cpu==1.7.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
model_path = "/content/drive/MyDrive/Colab_Notebooks/NLP/Project/models/medical_chatbot.pkl"
!cp -R $model_path .

In [4]:
import cloudpickle

In [5]:
cloudpickle.__version__

'2.2.1'

In [6]:
def load_model():
    with open("medical_chatbot.pkl", 'rb') as f:
        chain = cloudpickle.load(f)
    return chain

model = load_model()

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [7]:
def format_answer(text):
  # Replace the '<pad>' tag with an empty string
  cleaned_text = text.replace('<pad>', '')

  # Split the text into lines, strip leading and trailing whitespace, and join back into a single string
  cleaned_text = '\n'.join(line.strip() for line in cleaned_text.split('\n'))

  # Replace multiple spaces with a single space within each line
  cleaned_text = '\n'.join(' '.join(line.split()) for line in cleaned_text.split('\n'))


  return cleaned_text

In [8]:
questionnaire_path = "/content/drive/MyDrive/Colab_Notebooks/NLP/Project/datasets/Questionnaire.xlsx"
!cp $questionnaire_path .

In [9]:
import pandas as pd

In [10]:
df = pd.read_excel('Questionnaire.xlsx')
df['Answer'] = None
df.head()

,SN,Question,Disease,Answer
0,1.0,I'm worried about my friend. He's been having ...,Multiple Myeloma,None
1,NaN,Can you tell me other symptoms about the disease,NaN,None
2,NaN,Can you tell me about the diagnosis procedure?,NaN,None
3,NaN,Can you tell the treatment procedure?,NaN,None
4,NaN,Can you tell me the possible causes of the dis...,NaN,None


In [11]:
import math

def ask_chatbot(row):
  if not math.isnan(row['SN']):
    print(f"Begin Conversation {row.SN}")
    model.memory.clear()

  if row['Answer'] is None:
    question = row['Question']
    response = model({"question": question})
    answer = format_answer(response['answer'])
    print(f"Question: \n {question} \n Answer: \n {answer}")
    row['Answer'] = answer

  return row

In [12]:
eval_dataset = df.apply(lambda row: ask_chatbot(row), axis = 1)

Begin Conversation 1.0


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Question: 
 I'm worried about my friend. He's been having a lot of bone pain lately, especially in his back and hips, and he even fractured a bone with very little impact. Could it be something serious? 
 Answer: 
 Yes, bone pain and fractures can be serious and should be evaluated by a healthcare professional. It's important to note that fractures can be caused by a variety of conditions, including osteoporosis, avascular necrosis, and other musculoskeletal disorders. It's important to note that the severity of the bone pain and fracture should be evaluated by a healthcare professional to determine the appropriate course of treatment.

Question: 
 Can you tell me other symptoms about the disease 
 Answer: 
 Some other symptoms of osteoporosis include:
* Bone pain and tenderness in the affected area
* Muscle weakness and fatigue
* Waddling gait and difficulty climbing stairs or standing up from a chair
* Bone and muscle tenderness on pressure
* Fractures of the ribs and pelvis
* Osteom

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Question: 
 Can you tell me more about the disease? 
 Answer: 
 Pulmonary tuberculosis (TB) is a type of tuberculosis that affects the lungs. It is caused by the infection with Mycobacterium tuberculosis (MTB), which is part of a complex of organisms including M. bovis (reservoir cattle) and M. africanum (reservoir humans). Pulmonary TB can present as a self-limiting febrile illness, but clinical disease only occurs if there is a hypersensitivity reaction or progressive infection. Primary pulmonary TB commonly entails disease in extrathoracic as well as pulmonary or pleural sites. It is a common cause of lung damage and is the most common worldwide.



...

Question: 
 Does he need medical assitance? 
 Answer: 
 Yes, your friend may need medical assistance. She has been experiencing chest pain, shortness of breath, and a feeling of heaviness in her chest. She has no past medical history and no family history, and her only medication is the combined oral contraceptive pill. On examination, her vital signs are normal (respiratory rate 19 breaths/ min, oxygen saturations 98% on air, blood pressure 115/60 mmHg, heart rate 90 beats/min, temperature 37.5°C) and the physical examination is also normal. However, if her symptoms persist or worsen, she should seek medical assistance immediately.



/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Question: 
 Are there any home remedies? 
 Answer: 
 No, there are no home remedies for yellow fever. Yellow fever is a serious illness that requires prompt medical attention and treatment. Home remedies may not be effective in treating the disease and may even worsen the symptoms. It is important to seek medical attention if you experience any symptoms of yellow fever, including fever, headache, or other symptoms that are not responding to other treatments. It is also important to follow the instructions of your healthcare provider for proper treatment and care.



/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Question: 
 Willl he have long term effects? 
 Answer: 
 Yes, yellow fever can have long-term effects on the liver. The liver may become damaged and may not function properly for the rest of the person's life. This can lead to liver failure, which can cause chronic liver disease and other health problems. It's important to note that yellow fever is a serious illness and should be treated by a healthcare professional as soon as possible. If your friend has been diagnosed with yellow fever, they should receive appropriate treatment to prevent complications and improve their overall health.



In [13]:
# model.memory.clear()

In [14]:
eval_dataset

,SN,Question,Disease,Answer
0,1.0,I'm worried about my friend. He's been having ...,Multiple Myeloma,"Yes, bone pain and fractures can be serious an..."
1,NaN,Can you tell me other symptoms about the disease,NaN,Some other symptoms of osteoporosis include:\n...
2,NaN,Can you tell me about the diagnosis procedure?,NaN,The diagnostic procedure for osteoporosis is t...
3,NaN,Can you tell the treatment procedure?,NaN,The treatment procedure for osteoporosis depen...
4,NaN,Can you tell me the possible causes of the dis...,NaN,The possible causes of osteoporosis include ag...
5,2.0,My friend has been experiencing persistent cou...,Tuberculosis,It's important to note that persistent coughin...
6,NaN,Can you tell me more about the disease?,NaN,Pulmonary tuberculosis (TB) is a type of tuber...
7,NaN,Can you tell me about it is diagnosed?,NaN,Pulmonary tuberculosis (TB) is diagnosed throu...
8,NaN,Is there any treatment procedure?,NaN,The treatment procedure for pulmonary tubercul...
9,NaN,Do I need to visit the doctor quickly?,NaN,"Yes, it is important to visit the doctor if yo..."


In [15]:
answers = 'Answers.xlsx'
eval_dataset.to_excel(answers, index=False)

In [17]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [18]:
ans_path = "/content/drive/MyDrive/Colab_Notebooks/NLP/Project/datasets/"
!cp $answers $ans_path

In [ ]:
prompt_question = "I have a cold and nosebleed. What should I do?"
answer = model({"question":prompt_question})

In [ ]:
answer['chat_history']

In [ ]:
format_answer(answer['answer'])

In [ ]:
prompt_question = "Can I just treat myself at home?"
answer = model({"question":prompt_question})

In [ ]:
format_answer(answer['answer'])

In [ ]:
answer['chat_history']

In [ ]:
format_answer(answer['answer'])

In [ ]:
prompt_question = "My friend is choking. What should I do?"
answer = model({"question":prompt_question})

In [ ]:
answer['chat_history']

In [ ]:
format_answer(answer['answer'])